# Playing the game

This notebook provides companion code for my video on The Kelly Criterion. The video and instructions below should be enough to play around. Enjoy! 

In [1]:
import numpy as np

from betting_strategies import ConstantDollar, BetLikeADummy, Kelly, KellyWithAnEstimate
import altair as alt

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

The classes `ConstantDollar`, `BetLikeADummy`, `Kelly` and `KellyWithAnEstimate` are all subclasses of `BaseGame` and provide different types of strategy. If you want to get fancy, you could implement your own classes by overriding the `_strategy` method. If you'd like a deep understanding, read the code - I've tried to comment it to make it understandable.

We can start by inspecting the arguments of the ConstantDollar game (all inits are the same)

In [2]:
help(ConstantDollar.__init__)

Help on function __init__ in module betting_strategies:

__init__(self, prob_heads: float, payout_ratio: float, N_flips: int, N_games: int, initial_wealth: float = 50)
    Initialization for all BaseGame-s - don't override it! The game is determined by the probability of heads,
    the payout ratio of payout-to-wager for a bet on heads and the number of flips.
    
    Note : Without lose of generaltiy, we assume every wager is on heads. This is because the game is such that it's
    always best to bet on one outcome every time.
    
    Parameters
    ----------
    prob_heads : true probability of heads on each flip
    payout_ratio : if you wager wager-dollars on a flip for heads, you'll receive wager*payout_ratio, otherwise
    you'll lose wager.
    N_flips : The number of flips involved in one play of the game.
    N_games : The number of games you'll play to test your strategy
    initial_wealth : The amount of wealth you begin each game with.



In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [4]:
prob_heads = 0.7
payout_ratio = 4 / 6
N_flips = 100
N_games = 1000  # Careful - if this number is too high, things can get slow
initial_wealth = 50

args = dict(
    prob_heads=prob_heads,
    payout_ratio=payout_ratio,
    N_flips=N_flips,
    N_games=N_games,
    initial_wealth=initial_wealth,
)

constant_dollar = ConstantDollar(**args)
dummy = BetLikeADummy(**args)
kelly = Kelly(**args)
kelly_w_estimate = KellyWithAnEstimate(**args)

<IPython.core.display.Javascript object>

### Constant Dollar Wager

In [5]:
constant_dollar.amount = (
    5  # You can use this to change the constant dollar wage - here it's 5
)
constant_dollar.plot_games(n_games=25, log=False, opacity=0.5)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [6]:
constant_dollar.plot_growth_rate_distribution(
    n_games=N_games, min_max_growth_rate=[-0.04, 0.04], step_size=0.002
)

alt.Chart(...)

<IPython.core.display.Javascript object>

# Betting big when seeing a string of tails!

In [7]:
dummy.plot_games(n_games=50, log=False, opacity=0.5)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [8]:
dummy.plot_growth_rate_distribution(
    n_games=N_games, min_max_growth_rate=[-0.04, 0.04], step_size=0.002
)

alt.Chart(...)

<IPython.core.display.Javascript object>

# Kelly Betting

In [9]:
kelly.plot_games(n_games=25, log=True, opacity=0.5)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [10]:
kelly.plot_growth_rate_distribution(
    n_games=N_games, min_max_growth_rate=[-0.1, 0.1], step_size=0.002
)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [11]:
kelly.plot_exp_growth_rates_by_perc_wager()

alt.LayerChart(...)

<IPython.core.display.Javascript object>

# Kelly Betting with an Estimate

In [12]:
kelly_w_estimate.plot_games(n_games=25, log=True, opacity=0.5)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [13]:
kelly_w_estimate.plot_growth_rate_distribution(
    n_games=N_games, min_max_growth_rate=[-0.1, 0.1], step_size=0.002
)

alt.Chart(...)

<IPython.core.display.Javascript object>

# Kelly Betting with a Bayesian Estimate

In [14]:
from dataclasses import dataclass

import pandas as pd


@dataclass 
class Beta: 
    """Class to represent a Beta distribution, the conjugate prior for binomial model. 
    
    More info on conjugate priors here: https://en.wikipedia.org/wiki/Conjugate_prior
    
    """
    alpha: float 
    beta: float 

    def __post_init__(self) -> None: 
        assert 0 < self.alpha, "Alpha must be positive"
        assert 0 < self.beta, "Beta must be positive"

    @property 
    def mean(self) -> float: 
        return self.alpha / (self.alpha + self.beta)

    def conjugate_posterior(self, n: int, x: int) -> "Beta": 
        alpha_post = self.alpha + x 
        beta_post = self.beta + n - x 

        return Beta(alpha=alpha_post, beta=beta_post)

    def plot_distribution(self, n_samples: int = 1_000) -> alt.Chart: 
        df_samples = pd.DataFrame({"samples": np.random.beta(self.alpha, self.beta, size=n_samples)})

        return alt.Chart(df_samples).mark_bar().encode(
            alt.X("samples:Q", bin=alt.Bin(extent=[0, 1], step=0.05)), 
            y="count()"
        )

<IPython.core.display.Javascript object>

In [15]:
from betting_strategies import BaseGame

from typing import Union


def create_bayes_strategy(prior: Beta, wait_time: Union[int, None] = 10): 
    """Return class that 
    
    Args: 
        prior: Prior distribution for estimating the probability of heads
        wait_time: Optional wait time before calculating the estimate

    Returns: 
        Child class of BaseGame that allows for a Bayesian estimate for the Kelly strategy

    """
    class KellyWithBayesEstimate(BaseGame): 
        def _strategy(self, flips_so_far: np.ndarray, W: float) -> float:
            x = flips_so_far.sum()
            n = len(flips_so_far)

            if wait_time is not None and n < wait_time: 
                return 0

            p = prior.conjugate_posterior(n, x).mean   

            return W * self.kelly_bet(p, self.payout_ratio)

    return KellyWithBayesEstimate

<IPython.core.display.Javascript object>

### Uniform Prior

The probability of heads is unknown before the games. This can be represented with a beta prior that is equivalent to a uniform distribution on the (0, 1) interval.

In [16]:
prior = Beta(1, 1)
prior.plot_distribution(n_samples=10_000)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [17]:
kelly_w_bayes_estimate = create_bayes_strategy(prior, wait_time=10)(**args)

<IPython.core.display.Javascript object>

In [18]:
kelly_w_bayes_estimate.plot_games(n_games=25, log=True, opacity=0.5)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [19]:
kelly_w_bayes_estimate.plot_growth_rate_distribution(
    n_games=N_games, min_max_growth_rate=[-0.1, 0.1], step_size=0.002
)

alt.Chart(...)

<IPython.core.display.Javascript object>

Since bayesian methods help with low amounts of data, let's not wait to bet money and make use of our prior information

In [20]:
kelly_w_bayes_estimate_no_wait = create_bayes_strategy(prior, wait_time=None)(**args)

kelly_w_bayes_estimate_no_wait.plot_games(n_games=25, log=True, opacity=0.5)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [21]:
kelly_w_bayes_estimate_no_wait.plot_growth_rate_distribution(
    n_games=N_games, min_max_growth_rate=[-0.1, 0.1], step_size=0.002
)

alt.Chart(...)

<IPython.core.display.Javascript object>

### Informed Prior on the Coin

Might be likely that the coin is still fair but with a higher chance of being biased toward heads

In [22]:
prior = Beta(3, 2)
prior.plot_distribution()

alt.Chart(...)

<IPython.core.display.Javascript object>

In [23]:
kelly_w_bayes_estimate_informed = create_bayes_strategy(prior, wait_time=10)(**args)

<IPython.core.display.Javascript object>

In [24]:
kelly_w_bayes_estimate_informed.plot_games(n_games=25, log=True, opacity=0.5)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [25]:
kelly_w_bayes_estimate_informed.plot_growth_rate_distribution(
    n_games=N_games, min_max_growth_rate=[-0.1, 0.1], step_size=0.002
)

alt.Chart(...)

<IPython.core.display.Javascript object>

Let's no longer wait to bet and make use of our prior information again

In [26]:
kelly_w_bayes_estimate_informed_no_wait = create_bayes_strategy(prior, wait_time=None)(**args)

<IPython.core.display.Javascript object>

In [27]:
kelly_w_bayes_estimate_informed_no_wait.plot_games(n_games=25, log=True, opacity=0.5)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [28]:
kelly_w_bayes_estimate_informed_no_wait.plot_growth_rate_distribution(
    n_games=N_games, min_max_growth_rate=[-0.1, 0.1], step_size=0.002
)

alt.Chart(...)

<IPython.core.display.Javascript object>

## Comparison

In [29]:
strategies = {
    "dummy": dummy, 
    "constant": constant_dollar, 
    "frequentist": kelly_w_estimate, 
    "bayes": kelly_w_bayes_estimate, 
    "bayes_no_wait": kelly_w_bayes_estimate_no_wait, 
    "bayes_informed": kelly_w_bayes_estimate_informed, 
    "bayes_informed_no_wait": kelly_w_bayes_estimate_informed_no_wait,
    "theorectical": kelly, 
}

data = pd.DataFrame({
    name: strategy.simulate_growth_rates(N_games) for name, strategy in strategies.items()
}).melt()
data.columns = ["Strategy", "Growth Rate"]

<IPython.core.display.Javascript object>

In [30]:
bounds = {
    "lower": -0.1, 
    "upper": 0.15
}

df_plot = data.assign(value=lambda row: row["Growth Rate"].clip(**bounds))

boxplot = (
    alt.Chart(df_plot)
    .mark_boxplot()
    .encode(
        y=alt.Y("Strategy:O", sort=list(strategies.keys())), 
        x=alt.X("value:Q", title="Growth Rate (Clipped)")
    )
)
line = alt.Chart(pd.DataFrame({"x": [0]})).mark_rule(strokeDash=[10, 10]).encode(x="x")

boxplot + line

alt.LayerChart(...)

<IPython.core.display.Javascript object>

The Bayesian versions of kelly betting appear to hedge against extreme losses as compared to the frequentist approach. That is, the boxplot for `frequentist` has simulation outcomes with growth rate being less than -6% whereas each all the Bayesian methods do not. 

Interestingly enough, it appears hedge even more by not waiting to collect data and just bet from the start. That can be seen with the whiskers ending around -2% as opposed to -4% with the `bayes` strategy to `bayes_no_wait`. There is a similar ~1% change from `bayes_informed` to `bayes_informed_no_wait`

Compared to the theoretical kelly strategy, the Bayesians ones do not have as many positive growth rates. But the percent from the Bayesians are all slightly better than the frequentist. The extreme positive values on from the theoretical are not as large as some of the Bayesian methods but that is likely just random; The right end of the whisker shown in the boxplot is likely a more stable statistic as is still larger in the theoretical strategy. 

Overall, the median outcomes don't appear much different from each other from frequentist to Bayesian strategies. However, they appear to slightly increase the probability of have a postive growth and be beneficial for hedging extreme losses. 